# Начало отчета

In [ ]:

# %%javascript
# IPython.notebook.kernel.execute('ipynb_file_name_ext = "' + IPython.notebook.notebook_name + '"')
# // получить имя файла не питона а юпитер ноут в переменную ipynb_file_name
# // работает только когда запускаем вручную 

# // или
 
# %%javascript
# IPython.notebook.kernel.execute(`ipynb_file_name_ext = '${window.document.getElementById("notebook_name").innerHTML}'`);
# // получить имя файла не питона а юпитер ноут в переменную ipynb_file_name
# // работает только когда запускаем вручную 

In [ ]:
# ipynb_file_name_ext

In [ ]:
# задаем имя файла если жаваскрипт отработал криво а он нифига не пашет (( только если руками запускать а если ран алл то все криво работает ((
ipynb_file_name = 'Список оформленных заказов'
# ipynb_file_name

In [ ]:
# magic
#################################
%matplotlib inline 
#################################

# run magic в юпитере! НЕ РАБОТАЕТ в .py файлах
# run files исполнить файлы магией тут
%run /srv/jhub/share/._Additions/import_my_modules.py

# %run /srv/jhub/share/._Additions/import_libs.py
# %run /srv/jhub/share/._Additions/prints_.py
# %run /srv/jhub/share/._Additions/my_script.py этот импортится внутри import_my_modules.py
# %run /srv/jhub/share/._Additions/Connecting_.py
# %run /srv/jhub/share/._Additions/Date_gen.py
# %run /srv/jhub/share/._Additions/xx.py

#################################
# еще вариант magic который !РАБОТАЕТ в .py файлах
# get_ipython().run_line_magic('matplotlib', 'inline')
# run files
# get_ipython().run_line_magic('run', '/srv/jhub/share/._Additions/import_libs.py')
# get_ipython().run_line_magic('run', '/srv/jhub/share/._Additions/Connecting_.py')
# get_ipython().run_line_magic('run', '/srv/jhub/share/._Additions/my_script.py')
# get_ipython().run_line_magic('run', '/srv/jhub/share/._Additions/Date_gen.py')
# get_ipython().run_line_magic('run', '/srv/jhub/share/._Additions/xx.py')
############################## End ##############################
# help(Prints_), print(JiraTaskPrint.__doc__)

## Жмем сочетание клавиш:  Alt-Ctrl-L (на англ. раскладке)

In [ ]:
# выводим спарсенные данные с жиры описание и тд....
task_link = '''
https://jira.e2e4.ru/browse/BA-\
288
'''
JiraTaskPrint().print_tasks(task_link)

In [ ]:
# spl1 = '1000-01-01 00:00:00'
# spl2 = '1000-01-02 00:00:00'
# spl3 = '1000-01-03 00:00:00'

In [ ]:
# #  BA-288 старый под старое ТЗ 
# sql_query_1 = ''' 
#          /* 
#          * BA-288 Список оформленных заказов
#          * https://confluence.e2e4.ru/pages/viewpage.action?pageId=183326070
#          * https://jira.e2e4.ru/browse/BA-288
#          * */

#         with PARAMS as materialized (
#             select 
#         --		('01.09.2021' || ' 00:00:00')::timestamp date_start -- 30232-1 тут и 30241 PAID_ORDERS
#         --		,('30.09.2021' || ' 23:59:59')::timestamp date_end
#         --		 ( '\$date_start' || ' 00:00:00')::timestamp date_start 
#         -- 		,( '\$date_end' || ' 23:59:59')::timestamp date_end 
#                 ( 'DATE_START_replce' || ' 00:00:00')::timestamp    date_start 
#                 ,( 'DATE_END_replce' || ' 23:59:59')::timestamp    date_end
#             )

#         , PAID_ORDERS as (
#         select distinct
#             ai.tm_created "Дата создания заказа" --время Дата создания заказа
#             ,a.id "ID заказа" --позиции клиентских заказов
#             ,state.name "Cтатус заказа" -- orders.states справочник состояний клиентского заказа
#         -- 	,case when c.id is null then null else
#         --		concat('<a href="http://opentech.local/index.php?p=customers&id=',c.id,']">',c.name,'</a>') end "Клиент (Покупатель)" /*confluence*/
#                 --c.id "ID клиента" /*jupyter*/ 
#                 --c.name "Клиент c.name" /*jupyter*/
#             ,round(orders.get_order_summa_brutto(a.id),2) "Стоимость оформленных позиций" --вычисление общей суммы заказа COALESCE(SUM(a.price_net), 0)
#             ,pa.payment_id "pa.payment_id"
#             ,case when op.type=payments.type('CASHLESS') and op.date_out is not null then -pa.sum else pa.sum end "pa.sum" /*orders get_order_paid_sum*/
#         --	,oplatforms.id --not in (3921) --3921	ASUP	Магазин

#         from PARAMS _
#         cross join opentech.account_item ai
#         /*Сайт старый новый, Маркетплейс, Платформа, Площадка*/
#         left join orders.platforms oplatforms on oplatforms.id = ai.id_platform
#         join orders.states state on state.id = ai.state
#         join opentech.accounts a on a.id = ai.account_id
#         join opentech.customers c on c.id=a.id_customer
#         left join opentech.payments_account pa on pa.account_id = a.id
#         left join opentech.payments op on op.id = pa.payment_id 

#         where
#             oplatforms.id not in (3921) --3921	ASUP	Магазин
#             and a.state != 2 -- или a.del=0 /*это удалена или нет позиция лучше смотреть по состоянию a.state != 2*/
#             and ai.tm_created::date between date_start and date_end --время Дата создания заказаия заказа
#             /*confluence*/
#         --	and ai.tm_created::date between '\$date_start' and '\$date_end' --время Дата создания заказаия заказа
#         --	and case when 666=\$ordersatate then true else state.id=\$ordersatate end
#             /*jupyter*/
#         --	ai.tm_placed::date between date_start and date_end --время Время оформления заказа	
#             /*tests*/
#         --	ai.siteuserid is not null --учетная запись сайта, с которой был оформлен заказ
#         --	and a.user_login = 'client' --кто добавил позицию в заказ
#         --	and 
#         --	a.id in ( 
#         --6264551,6285751,6287828,6248596,6252709,6257513,6257527,6258729,6264196,6266103,6274150,6295375,6301567
#         --		)
#         )
#         --select * from PAID_ORDERS

#         --сумма стоимости всех позиц в статусе продано (Стоимость отгруженных позиций) 
#         , COST_OF_ITEMS_SHIPPED as (
#         select --distinct
#             a.id "ID заказа" --позиции клиентских заказов
#         -- 	case when a.id is null then null else --позиции клиентских заказов
#         ---- 		concat('<a href="http://opentech.local/new/order/',a.id,'?main=2&additional=8">',a.id,'</a>') end "ID заказа" /*confluence*/  
#         -- 		concat( '=HYPERLINK("http://opentech.local/new/order/',a.id,'", "',a.id,'")' ) end "ID заказа" /*jupyter*/ --excel
#             ,sum(a.price_net) "Стоимость отгруженных позиций" --сумма всех позиц (продажная) в статусе продано 
#         from PARAMS _
#         cross join opentech.account_item ai
#         /*Сайт старый новый, Маркетплейс, Платформа, Площадка*/
#         left join orders.platforms oplatforms on oplatforms.id = ai.id_platform
#         join orders.states state on state.id = ai.state
#         join opentech.accounts a on a.id = ai.account_id
#         left join opentech.store os ON a.store_id = os.id
#         --/* место продажи */
#         join opentech.storage storage on storage.id = a.storage_id
#         join public.cities pc on pc.id = storage.city_id 

#         where
#             oplatforms.id not in (3921)
#         --	and a.state != 2 -- или a.del=0 /*это удалена или нет позиция лучше смотреть по состоянию a.state != 2*/
#             and a.state = 23 --по стат продано (Состояние позиции заказа) --которые купили товары или услуги в статусе списания "Продано"
#             and ai.tm_created::date between date_start and date_end --время Дата создания заказаия заказа
#             /*confluence*/
#         --	and ai.tm_created::date between '\$date_start' and '\$date_end' --время Дата создания заказаия заказа
#         --	and case when 0=\$city then true else pc.id=\$city end
#         --	and case when 0=\$ordersatate then true else state.id=\$ordersatate end
#             /*jupyter*/
#         --	and ai.tm_placed::date between date_start and date_end --время Время оформления заказа
#             /*tests*/
#         --	ai.siteuserid is not null --учетная запись сайта, с которой был оформлен заказ 
#         --	and a.user_login = 'client' --кто добавил позицию в заказ
#         --	and 
#         --	a.id in ( 
#         --6264551,6285751,6287828,6248596,6252709,6257513,6257527,6258729,6264196,6266103,6274150,6295375,6301567
#         --		)

#         group by a.id
#         )
#         --select * from COST_OF_ITEMS_SHIPPED

#         , ORDER_VALUES as (
#         select distinct
#             ai.tm_created "Дата создания заказа" --время Дата создания заказаия заказа
#             ,state.name "Cтатус заказа" -- orders.states справочник состояний клиентского заказа
#         -- 	,case when c.id is null then null else
#         --		concat('<a href="http://opentech.local/index.php?p=customers&id=',c.id,']">',c.name,'</a>') end "Клиент (Покупатель)" /*confluence*/
#             --c.id "ID клиента" /*jupyter*/ 
#             ,c.name "Клиент (Покупатель)" /*jupyter*/
#             ,array_to_string(array_agg(distinct pc.name),'; ') "Город отгрузки"
#             ,a.id "ID заказа" 
#         -- 	,case when a.id is null then null else 
#         ---- 		concat('<a href="http://opentech.local/new/order/',a.id,'?main=2&additional=8">',a.id,'</a>') end "ID заказа" /*confluence*/ 
#         -- 		concat( '=HYPERLINK("http://opentech.local/new/order/',a.id,'", "',a.id,'")' ) end "ID заказа" /*jupyter*/ --excel
#             ,sum(coalesce(a.price_net,0)) "Стоимость оформленных позиций"
#         --	,round(orders.get_order_summa_brutto(a.id),2) "Стоимость оформленных позиций" --вычисление общей суммы заказа
#             ,sum(coalesce(os.sprice,0)) "Чистая себестоимость позиций"
#             ,op.name "Площадка" --	,ai.id_platform -- Сайт старый новый, Маркетплейс, Платформа, Площадка
#             ,array_to_string(array_agg(distinct sold_state.name),'; ') "Статус списания ФОН" --store.shipping_states sold_state справочник состояний клиентского заказа
#             ,array_to_string(array_agg(distinct ois.name),'; ') "Статус списания позиции" -- Состояние позиции заказа
#             ,array_to_string(array_agg(distinct case -- 'Подарочная карта' 'Доставка' 'Услуга' 'Товар' 'Комплектующая'
#                 when pg.id_payment is not null then 'Подарочная карта'
#                 when a.correct_name ilike '%оставка%' then 'Доставка'
#                 when os2.type is not null then 'Услуга'
#                 when a.state != orders.item_state('BUNDLED') then 'Товар'
#                 when a.state = orders.item_state('BUNDLED') then 'Комплектующая'
#                 end),'; ') "Тип позиции"
#         --	,a.reference_id	"Товарная позиция" 

#         from PARAMS _
#         cross join opentech.account_item ai
#         /*Сайт старый новый, Маркетплейс, Платформа, Площадка*/
#         left join orders.platforms op on op.id = ai.id_platform
#         join orders.states state on state.id = ai.state
#         join opentech.accounts a on a.id = ai.account_id
#         join opentech.customers c on c.id=a.id_customer
#         left join opentech.store os ON a.store_id = os.id
#         /* место продажи */
#         join opentech.storage storage on storage.id = a.storage_id
#         join public.cities pc on pc.id = storage.city_id 
#         left join store.shipping_states sold_state ON sold_state.id = os.sold
#         /*тип позиции (фон)*/
#         left join payments.giftcards pg on a.i = pg.id_order_item 
#         left join opentech.orderservices os2 on  a.i = os2.orderitemid
#         left join orders.item_states ois on ois.id = a.state

#         where
#             op.id not in (3921) --3921	ASUP	Магазин
#             and a.state != 2 -- или a.del=0 /*это удалена или нет позиция лучше смотреть по состоянию a.state != 2*/
#             and ai.tm_created::date between date_start and date_end --время Дата создания заказаия заказа
#             /*confluence*/
#         --	and ai.tm_created::date between '\$date_start' and '\$date_end' --время Дата создания заказаия заказа
#         --	and case when 0=\$city then true else pc.id=\$city end
#         --	and case when 666=\$ordersatate then true else state.id=\$ordersatate end
#         --	and case when 0=\$ordersplatform then true else op.id=\$ordersplatform end
#             /*jupyter*/
#         --	and ai.tm_placed::date between date_start and date_end --время Время оформления заказа
#             /*tests*/	
#         --	ai.siteuserid is not null --учетная запись сайта, с которой был оформлен заказ
#         --	a.user_login = 'client' --кто добавил позицию в заказ 
#         --	and 
#         --	a.id in ( 
#         --6264551,6285751,6287828,6248596,6252709,6257513,6257527,6258729,6264196,6266103,6274150,6295375,6301567
#         --		)

#         group by a.id, ai.tm_created, state.name, c.id, op.name

#         )
#         --select * from ORDER_VALUES
#         --order by "ID заказа"

#         , end_all as materialized (
#         select 
#         --	array_to_string(array_agg(distinct o_v."Город отгрузки"),'; ') "Город отгрузки"
#             o_v."Город отгрузки"
#             ,o_v."Дата создания заказа"::date::text 
#         --	,o_v."ID заказа"::text 
#             ,case when o_v."ID заказа" is null then null else --позиции клиентских заказов
#         -- 		concat('<a href="http://opentech.local/new/order/',o_v."ID заказа",'?main=2&additional=8">',o_v."ID заказа",'</a>') end "ID заказа" /*confluence*/  
#                 concat( '=HYPERLINK("http://opentech.local/new/order/',o_v."ID заказа",'", "',o_v."ID заказа",'")' ) end "ID заказа" /*jupyter*/ 

#             ,o_v."Cтатус заказа"::text 
#             ,o_v."Клиент (Покупатель)"::text 
#         --	,case when max(p_o."pa.payment_id") is null then null 
#         --		else array_to_string(array_agg(distinct concat(
#         --			'<a href="http://opentech.local/payments/',p_o."pa.payment_id",'">',p_o."pa.payment_id",'</a>'
#         --			)),'; ') end "ID платежей" /*confluence*/
#             ,array_to_string(array_agg(concat(p_o."pa.payment_id")),'; ') "ID платежей" /*jupyter*/
#             ,array_to_string(array_agg( round(p_o."pa.sum",2)),'; ') "Платежи по заказу"
#             ,sum(p_o."pa.sum") "Оплачено по заказу"
#             ,p_o."Стоимость оформленных позиций"
#             ,case when c_i_s."Стоимость отгруженных позиций" = 0 then null else c_i_s."Стоимость отгруженных позиций" end
#             ,case when o_v."Чистая себестоимость позиций" = 0 then null else o_v."Чистая себестоимость позиций" end 
#             /*% выкупа по оплате - стоимость оплаченных позиций/стоимость всех оформленных позиций*/
#             ,case 
#                 when sum(p_o."pa.sum") = any(array[null,0]) then null
#                 when p_o."Стоимость оформленных позиций" <> any(array[null,0]) /*деление на ноль*/
#                     then round(( sum(p_o."pa.sum") / p_o."Стоимость оформленных позиций")*100,0)::text
#                 else null end "% выкупа по оплате"
#             /*% выкупа по отгрузке - стоимость отгруж позиций/стоимость всех оформленных позиций*/
#             ,case 
#                 when c_i_s."Стоимость отгруженных позиций" = any(array[null,0]) then null 
#                 when p_o."Стоимость оформленных позиций" <> any(array[null,0]) /*деление на ноль*/
#                     then round(( c_i_s."Стоимость отгруженных позиций" / p_o."Стоимость оформленных позиций")*100,0)::text
#                 else null end "% выкупа по отгрузке"		
#             ,c_i_s."Стоимость отгруженных позиций" - o_v."Чистая себестоимость позиций" "Валовый доход по ЧС" --должно быть сумма отгруженных позиций - чистая себестоимость = валовый доход по ЧС
#             ,o_v."Площадка"
#             ,o_v."Статус списания позиции"
#             ,o_v."Статус списания ФОН"
#             ,o_v."Тип позиции" "Тип позиции"

#         from ORDER_VALUES o_v
#         --Дата создания заказа ID заказа Cтатус заказа Клиент (Покупатель)
#         --"Платежи по заказу"	"Оплачено по заказу"	"Стоимость оформленных позиций"	"Стоимость отгруженных позиций"	"Чистая себестоимость позиций"
#         join PAID_ORDERS p_o on p_o."ID заказа" = o_v."ID заказа" 
#         --	and p_o."Стоимость оформленных позиций" = o_v."Стоимость оформленных позиций"
#         --	and p_o."Дата создания заказа" = o_v."Дата создания заказа"
#         --	and p_o."Cтатус заказа" = o_v."Cтатус заказа" 
#         --	and p_o."Клиент (Покупатель)" = o_v."Клиент (Покупатель)" 
#         left join COST_OF_ITEMS_SHIPPED c_i_s on p_o."ID заказа" = c_i_s."ID заказа"

#         group by o_v."ID заказа",o_v."Дата создания заказа",o_v."Cтатус заказа",o_v."Клиент (Покупатель)",p_o."Стоимость оформленных позиций",c_i_s."Стоимость отгруженных позиций"
#             ,o_v."Чистая себестоимость позиций",o_v."Площадка",o_v."Статус списания ФОН",o_v."Статус списания позиции",o_v."Тип позиции",o_v."Город отгрузки"
#         order by o_v."Дата создания заказа" --,o_v."ID заказа"
#         )

#         select  
#             row_number() OVER()::text "№", 
#             * 
#         from end_all e_a  	
#         --where /*костыль */
#         --	case when not 
#         --	(
#         --	e_a."Чистая себестоимость позиций" is null and e_a."Валовый доход по ЧС" is null
#         --	e_a."Оплачено по заказу" is not null and e_a."Стоимость оформленных позиций" is not null and e_a."Стоимость отгруженных позиций" is not null 
#         --	and e_a."Статус списания позиции" ilike ''		
#         --	and "Тип позиции" ilike 'Товар')
#         --	then true end

#         union all
#         select 
#             'ИТОГО' "№"
#             ,null "Город отгрузки"
#             ,null "Дата создания заказа"
#             ,null "ID заказа"
#             ,null "Cтатус заказа"
#             ,null "Клиент (Покупатель)"
#             ,null "ID платежей"
#             ,null "Платежи по заказу"
#             ,round(sum("Оплачено по заказу" ),2)
#             ,round(sum("Стоимость оформленных позиций" ),2)
#             ,round(sum("Стоимость отгруженных позиций" ),2)
#             ,round(sum("Чистая себестоимость позиций" ),2)
#             ,null "% выкупа по оплате"
#             ,null "% выкупа по отгрузке"
#             ,round(sum("Валовый доход по ЧС" ),2)
#             ,null "Площадка"
#             ,null "Статус списания позиции"
#             ,null "Статус списания ФОН"
#             ,null "Тип позиции"
#         from end_all e_a
#         group by 1,2,3,4,5,6,7,8,13,14,16,17,18
#     '''

In [ ]:
# # BA-453 тут касяк не то выдает 
# sql_query_1 = '''

# /* 26 257
#  * BA-288 BA-453 Список оформленных заказов
#  * https://confluence.e2e4.ru/pages/viewpage.action?pageId=183326070
#  * https://jira.e2e4.ru/browse/BA-288
#  * */

# with PARAMS as materialized (
#     select 
# --		('01.03.2023' || ' 00:00:00' )::timestamp date_start -- 30232-1 тут и 30241 PAID_ORDERS
# --		,('31.03.2023' || ' 23:59:59')::timestamp date_end
# --		 ( '\$date_start' || ' 00:00:00')::timestamp date_start 
# -- 		,( '\$date_end' || ' 23:59:59')::timestamp date_end 
#         ( 'DATE_START_replce' || ' 00:00:00')::timestamp    date_start 
#         ,( 'DATE_END_replce' || ' 23:59:59')::timestamp    date_end
#     )

# , PAID_ORDERS as (
# select distinct
#     ai.tm_created "Дата создания заказа" --время Дата создания заказа
#     ,ai.tm_placed "Время оформления заказа"
#     ,ai.account_id "ID заказа" --позиции клиентских заказов
#     ,state.name "Cтатус заказа" -- orders.states справочник состояний клиентского заказа
# -- 	,case when c.id is null then null else
# --		concat('<a href="http://opentech.local/index.php?p=customers&id=',c.id,']">',c.name,'</a>') end "Клиент (Покупатель)" /*confluence*/
#         --c.id "ID клиента" /*jupyter*/ 
#         --c.name "Клиент c.name" /*jupyter*/
#     ,round(orders.get_order_summa_brutto(ai.account_id),2) "Стоимость оформленных позиций" --вычисление общей суммы заказа COALESCE(SUM(a.price_net), 0)
#     ,pa.payment_id "pa.payment_id"
#     ,case when op.type=payments.type('CASHLESS') and op.date_out is not null then -pa.sum else pa.sum end "pa_sum" /*orders get_order_paid_sum*/
#     ,first_value(u."name") over (partition by a.id order by lai._values::hstore -> 'tm_placed' desc ) as last_user_logins
#     ,first_value(u.id) over (partition by a.id order by lai._values::hstore -> 'tm_placed' desc ) as u_id
#     ,first_value(lai._values::hstore -> 'tm_placed') over (partition by a.id order by lai._values::hstore -> 'tm_placed' desc ) as last_tm_placed       

# 	--	,oplatforms.id 
# 	--  ,a.user_login
# 	--  ,a.in_account_date

# from PARAMS _
# cross join opentech.account_item ai
# /*Сайт старый новый, Маркетплейс, Платформа, Площадка*/
# left join orders.platforms oplatforms on oplatforms.id = ai.id_platform
# join orders.states state on state.id = ai.state
# join opentech.accounts a on a.id = ai.account_id
# LEFT JOIN opentech.customers c on c.id=a.id_customer
# left join opentech.payments_account pa on pa.account_id = a.id
# left join opentech.payments op on op.id = pa.payment_id 
# left join _logs.opentech__account_item lai on lai.id = ai.id
# left join opentech.users u on u.id = lai._id_user
# /*left join opentech.users u on lower(u.login) = lower(a.user_login)*/


# where
#     oplatforms.id in ( 7832, 1246, 3252, 2576, 5804) --3921	ASUP	Магазин
#     and ai.tm_placed::date between date_start and date_end --время Дата создания заказаия заказа
#     /*and a.state != 2*/ -- или a.del=0 /*это удалена или нет позиция лучше смотреть по состоянию a.state != 2*/
#     /*confluence*/
# --	and ai.tm_placed::date between '\$date_start' and '\$date_end' --время Дата создания заказаия заказа
# --	and case when 666=\$ordersatate then true else state.id=\$ordersatate end
#     /*jupyter*/
# --	ai.tm_placed::date between date_start and date_end --время Время оформления заказа	
#     /*tests*/
# --	ai.siteuserid is not null --учетная запись сайта, с которой был оформлен заказ
# --	and a.user_login = 'client' --кто добавил позицию в заказ
# --    and ai.state = 0
# --	and ai.account_id in ( 7005027)
# --	a.id in ( 
# --		6264551,6285751,6287828,6248596,6252709,6257513,6257527,6258729,6264196,6266103,6274150,6295375,6301567
# --		)
# --order by a.in_account_date
# )
# --select * from PAID_ORDERS


# --сумма стоимости всех позиц в статусе продано (Стоимость отгруженных позиций) 
# , COST_OF_ITEMS_SHIPPED as (
# select --distinct
#     a.id "ID заказа" --позиции клиентских заказов
# -- 	case when a.id is null then null else --позиции клиентских заказов
# ---- 		concat('<a href="http://opentech.local/new/order/',a.id,'?main=2&additional=8">',a.id,'</a>') end "ID заказа" /*confluence*/  
# -- 		concat( '=HYPERLINK("http://opentech.local/new/order/',a.id,'", "',a.id,'")' ) end "ID заказа" /*jupyter*/ --excel
#     ,sum(a.price_net) "Стоимость отгруженных позиций" --сумма всех позиц (продажная) в статусе продано 
# from PARAMS _
# cross join opentech.account_item ai
# /*Сайт старый новый, Маркетплейс, Платформа, Площадка*/
# left join orders.platforms oplatforms on oplatforms.id = ai.id_platform
# join opentech.accounts a on a.id = ai.account_id
# join orders.states state on state.id = ai.state
# left join opentech.store os ON a.store_id = os.id
# --/* место продажи */
# join opentech.storage storage on storage.id = a.storage_id
# join public.cities pc on pc.id = storage.city_id 

# where
#     oplatforms.id in ( 7832, 1246, 3252, 2576, 5804)
# 	/*and a.state != 2*/ -- или a.del=0 /*это удалена или нет позиция лучше смотреть по состоянию a.state != 2*/
#     and a.state = 23 --по стат продано (Состояние позиции заказа) --которые купили товары или услуги в статусе списания "Продано"
#     and ai.tm_placed::date between date_start and date_end --время Дата создания заказаия заказа
#     /*confluence*/
# --	and ai.tm_placed::date between '\$date_start' and '\$date_end' --время Дата создания заказаия заказа
# --	and case when 0=\$city then true else pc.id=\$city end
# --	and case when 0=\$ordersatate then true else state.id=\$ordersatate end
#     /*jupyter*/
# --	and ai.tm_placed::date between date_start and date_end --время Время оформления заказа
#     /*tests*/
# --	ai.siteuserid is not null --учетная запись сайта, с которой был оформлен заказ 
# --	and a.user_login = 'client' --кто добавил позицию в заказ
# --	and 
# --	a.id in ( 
# --6264551,6285751,6287828,6248596,6252709,6257513,6257527,6258729,6264196,6266103,6274150,6295375,6301567
# --		)
# group by a.id
# )
# --select * from COST_OF_ITEMS_SHIPPED



# , ORDER_VALUES as (
# select --distinct
#     ai.tm_created "Дата создания заказа" --время Дата создания заказаия заказа
#     ,ai.tm_placed "Время оформления заказа"
#     ,state.name "Cтатус заказа" -- orders.states справочник состояний клиентского заказа
# -- 	,case when c.id is null then null else
# --		concat('<a href="http://opentech.local/index.php?p=customers&id=',c.id,']">',c.name,'</a>') end "Клиент (Покупатель)" /*confluence*/
#     --c.id "ID клиента" /*jupyter*/ 
#     ,c.name "Клиент (Покупатель)" /*jupyter*/
#     ,array_to_string(array_agg(distinct pc.name),'; ') "Город отгрузки"
#     ,ai.account_id "ID заказа" 
# -- 	,case when a.id is null then null else 
# ---- 		concat('<a href="http://opentech.local/new/order/',a.id,'?main=2&additional=8">',a.id,'</a>') end "ID заказа" /*confluence*/ 
# -- 		concat( '=HYPERLINK("http://opentech.local/new/order/',a.id,'", "',a.id,'")' ) end "ID заказа" /*jupyter*/ --excel
#     ,sum(coalesce(a.price_net,0)) "Стоимость оформленных позиций"
# --	,round(orders.get_order_summa_brutto(a.id),2) "Стоимость оформленных позиций" --вычисление общей суммы заказа
#     ,sum(coalesce(os.sprice,0)) "Чистая себестоимость позиций"
#     ,op.name "Площадка" --	,ai.id_platform -- Сайт старый новый, Маркетплейс, Платформа, Площадка
#     ,array_to_string(array_agg(distinct sold_state.name),'; ') "Статус списания ФОН" --store.shipping_states sold_state справочник состояний клиентского заказа
#     ,array_to_string(array_agg(distinct ois.name),'; ') "Статус списания позиции" -- Состояние позиции заказа
#     ,array_to_string(array_agg(distinct case -- 'Подарочная карта' 'Доставка' 'Услуга' 'Товар' 'Комплектующая'
#         when pg.id_payment is not null then 'Подарочная карта'
#         when a.correct_name ilike '%оставка%' then 'Доставка'
#         when os2.type is not null then 'Услуга'
#         when a.state != orders.item_state('BUNDLED') then 'Товар'
#         when a.state = orders.item_state('BUNDLED') then 'Комплектующая'
#         end),'; ') "Тип позиции"
# --	,a.reference_id	"Товарная позиция" 

# from PARAMS _
# cross join opentech.account_item ai
# /*Сайт старый новый, Маркетплейс, Платформа, Площадка*/
# left join orders.platforms op on op.id = ai.id_platform
# join orders.states state on state.id = ai.state
# join opentech.accounts a on a.id = ai.account_id
# join opentech.customers c on c.id=a.id_customer
# left join opentech.store os ON a.store_id = os.id
# /* место продажи */
# join opentech.storage storage on storage.id = a.storage_id
# join public.cities pc on pc.id = storage.city_id 
# left join store.shipping_states sold_state ON sold_state.id = os.sold
# /*тип позиции (фон)*/
# left join payments.giftcards pg on a.i = pg.id_order_item 
# left join opentech.orderservices os2 on  a.i = os2.orderitemid
# left join orders.item_states ois on ois.id = a.state

# where
#     op.id in ( 7832, 1246, 3252, 2576, 5804) --3921	ASUP	Магазин
#     /*and a.state != 2*/ -- или a.del=0 /*это удалена или нет позиция лучше смотреть по состоянию a.state != 2*/
#     and ai.tm_placed::date between date_start and date_end --время Дата создания заказаия заказа
#     /*confluence*/
# --	and ai.tm_placed::date between '\$date_start' and '\$date_end' --время Дата создания заказаия заказа
# --	and case when 0=\$city then true else pc.id=\$city end
# --	and case when 666=\$ordersatate then true else state.id=\$ordersatate end
# --	and case when 0=\$ordersplatform then true else op.id=\$ordersplatform end
#     /*jupyter*/
# --	and ai.tm_placed::date between date_start and date_end --время Время оформления заказа
#     /*tests*/	
# --    and ai.state = 0
# --	and ai.account_id in ( 7005027)
# --	ai.siteuserid is not null --учетная запись сайта, с которой был оформлен заказ
# --	a.user_login = 'client' --кто добавил позицию в заказ 
# --	and 
# --	a.id in ( 
# --6264551,6285751,6287828,6248596,6252709,6257513,6257527,6258729,6264196,6266103,6274150,6295375,6301567
# --		)
# group by ai.account_id, ai.tm_created, ai.tm_placed, state.name, c.id, op.name

# )
# --select * from ORDER_VALUES
# --order by "ID заказа"

# , end_all as materialized (
# select 
# --	array_to_string(array_agg(distinct o_v."Город отгрузки"),'; ') "Город отгрузки"
#     o_v."Город отгрузки"
#     ,o_v."Дата создания заказа"::text 
#     ,o_v."Время оформления заказа"::text
# --	,o_v."ID заказа"::text 
#     ,case when o_v."ID заказа" is null then null else --позиции клиентских заказов
# -- 		concat('<a href="http://opentech.local/new/order/',o_v."ID заказа",'?main=2&additional=8">',o_v."ID заказа",'</a>') end "ID заказа" /*confluence*/  
#         concat( '=HYPERLINK("http://opentech.local/new/order/',o_v."ID заказа",'", "',o_v."ID заказа",'")' ) end "ID заказа" /*jupyter*/ 
#     ,o_v."Cтатус заказа"::text 
#     ,p_o.last_user_logins "Оформил"
#     ,p_o.last_tm_placed::text "Время оформления"
#     ,o_v."Клиент (Покупатель)"::text 
# --	,case when max(p_o."pa.payment_id") is null then null 
# --		else array_to_string(array_agg(distinct concat(
# --			'<a href="http://opentech.local/payments/',p_o."pa.payment_id",'">',p_o."pa.payment_id",'</a>'
# --			)),'; ') end "ID платежей" /*confluence*/
#     ,array_to_string(array_agg(concat(p_o."pa.payment_id")),'; ') "ID платежей" /*jupyter*/
#     ,array_to_string(array_agg( round(p_o."pa_sum",2)),'; ') "Платежи по заказу"
#     ,sum(p_o."pa_sum") "Оплачено по заказу"
#     ,p_o."Стоимость оформленных позиций"
#     ,case when c_i_s."Стоимость отгруженных позиций" = 0 then null else c_i_s."Стоимость отгруженных позиций" end
#     ,case when o_v."Чистая себестоимость позиций" = 0 then null else o_v."Чистая себестоимость позиций" end 
#     /*% выкупа по оплате - стоимость оплаченных позиций/стоимость всех оформленных позиций*/
#     ,case 
#         when sum(p_o."pa_sum") = any(array[null,0]) then null
#         when p_o."Стоимость оформленных позиций" <> any(array[null,0]) /*деление на ноль*/
#             then round(( sum(p_o."pa_sum") / p_o."Стоимость оформленных позиций")*100,0)::text
#         else null end "% выкупа по оплате"
#     /*% выкупа по отгрузке - стоимость отгруж позиций/стоимость всех оформленных позиций*/
#     ,case 
#         when c_i_s."Стоимость отгруженных позиций" = any(array[null,0]) then null 
#         when p_o."Стоимость оформленных позиций" <> any(array[null,0]) /*деление на ноль*/
#             then round(( c_i_s."Стоимость отгруженных позиций" / p_o."Стоимость оформленных позиций")*100,0)::text
#         else null end "% выкупа по отгрузке"		
#     ,c_i_s."Стоимость отгруженных позиций" - o_v."Чистая себестоимость позиций" "Валовый доход по ЧС" --должно быть сумма отгруженных позиций - чистая себестоимость = валовый доход по ЧС
#     ,o_v."Площадка"
#     ,o_v."Статус списания позиции"
#     ,o_v."Статус списания ФОН"
#     ,o_v."Тип позиции" "Тип позиции"

# from ORDER_VALUES o_v
# --Дата создания заказа ID заказа Cтатус заказа Клиент (Покупатель)
# --"Платежи по заказу"	"Оплачено по заказу"	"Стоимость оформленных позиций"	"Стоимость отгруженных позиций"	"Чистая себестоимость позиций"
# join PAID_ORDERS p_o on p_o."ID заказа" = o_v."ID заказа" 
# --	and p_o."Стоимость оформленных позиций" = o_v."Стоимость оформленных позиций"
# --	and p_o."Дата создания заказа" = o_v."Дата создания заказа"
# --	and p_o."Cтатус заказа" = o_v."Cтатус заказа" 
# --	and p_o."Клиент (Покупатель)" = o_v."Клиент (Покупатель)" 
# left join COST_OF_ITEMS_SHIPPED c_i_s on p_o."ID заказа" = c_i_s."ID заказа"

# group by 
# 	o_v."ID заказа",o_v."Дата создания заказа",o_v."Время оформления заказа",o_v."Cтатус заказа",p_o.last_user_logins ,o_v."Клиент (Покупатель)",p_o."Стоимость оформленных позиций",c_i_s."Стоимость отгруженных позиций"
#     ,o_v."Чистая себестоимость позиций",o_v."Площадка",o_v."Статус списания ФОН",o_v."Статус списания позиции",o_v."Тип позиции",o_v."Город отгрузки",p_o.last_tm_placed 
# order by o_v."Дата создания заказа" --,o_v."ID заказа"
# )

# select  
#     row_number() OVER()::text "№", 
#     * 
# from end_all e_a  	
# --where /*костыль*/
# --	case when not 
# --	(
# --	e_a."Чистая себестоимость позиций" is null and e_a."Валовый доход по ЧС" is null
# --	e_a."Оплачено по заказу" is not null and e_a."Стоимость оформленных позиций" is not null and e_a."Стоимость отгруженных позиций" is not null 
# --	and e_a."Статус списания позиции" ilike ''		
# --	and "Тип позиции" ilike 'Товар')
# --	then true end

# union all
# select 
#     'ИТОГО' "№"
#     ,null "Время оформления заказа"
#     ,null "Город отгрузки"
#     ,null "Дата создания заказа"
#     ,null "ID заказа"
#     ,null "Cтатус заказа"
#     ,null "Оформил"
#     ,null "Время оформления"
#     ,null "Клиент (Покупатель)"
#     ,null "ID платежей"
#     ,null "Платежи по заказу"
#     ,round(sum("Оплачено по заказу" ),2)
#     ,round(sum("Стоимость оформленных позиций" ),2)
#     ,round(sum("Стоимость отгруженных позиций" ),2)
#     ,round(sum("Чистая себестоимость позиций" ),2)
#     ,null "% выкупа по оплате"
#     ,null "% выкупа по отгрузке"
#     ,round(sum("Валовый доход по ЧС" ),2)
#     ,null "Площадка"
#     ,null "Статус списания позиции"
#     ,null "Статус списания ФОН"
#     ,null "Тип позиции"
# from end_all e_a
# group by 1,2,3,4,5,6,7,8,9,10,11, 16,17, 19,20,21  
        
# '''

In [ ]:
# # BA-453 оптимизировал 

# old 2023.06.20 sql_query_1 = '''

# /* 26 257
#  * BA-288 BA-453 Список оформленных заказов
#  * https://confluence.e2e4.ru/pages/viewpage.action?pageId=183326070
#  * https://jira.e2e4.ru/browse/BA-288
#  * */

# with PARAMS as materialized (
#     select 
# --		('01.03.2023' || ' 00:00:00' )::timestamp date_start 
# --		,('31.03.2023' || ' 23:59:59')::timestamp date_end
# --		 ( '\$date_start' || ' 00:00:00')::timestamp date_start 		/*confluence*/
# -- 		,( '\$date_end' || ' 23:59:59')::timestamp date_end 			/*confluence*/
#         ( 'DATE_START_replce' || ' 00:00:00')::timestamp    date_start	/*jupyter*/
#         ,( 'DATE_END_replce' || ' 23:59:59')::timestamp    date_end 		/*jupyter*/
#     )

# , GENERAL as ( 
# select distinct
# 	ai.id 
# 	,a.i
#     ,ai.tm_created "Дата создания заказа" --время Дата создания заказа
#     ,ai.tm_placed "Время оформления заказа"
#     /*,in_account_date "Время добавления позиции в заказ"*/
#     ,ai.account_id "ID заказа" --позиции клиентских заказов
# --	,a.id a_id
#     ,state.name "Cтатус заказа" -- orders.states справочник состояний клиентского заказа
# -- 	,case when c.id is null then null else
# --		concat('<a href="http://opentech.local/index.php?p=customers&id=',c.id,']">',c.name,'</a>') end "Клиент (Покупатель)" /*confluence*/
# --  ,c.id "ID клиента" /*jupyter*/ 
#     ,c.name "Клиент (Покупатель)" /*jupyter*/
#     ,pc.name "Город отгрузки"
#     ,case orders.get_order_summa_brutto(ai.account_id) when 0 then null 
#     	else round(orders.get_order_summa_brutto(ai.account_id),2) end "Стоимость оформленных позиций" --вычисление общей суммы заказа COALESCE(SUM(a.price_net), 0)        
#     ,oplatforms.name "Площадка" 
#     ,sold_state.name "Статусы списания товара ФОН" --store.shipping_states Статусы списания товара
#     ,ois.name "Статус списания позиции" -- Справочник состояний позиции клиентского заказа
#     ,a.state a_state
#     ,a.price_net price_net
#     ,a.price
#     ,os.sprice sprice
#     ,case -- 'Подарочная карта' 'Доставка' 'Услуга' 'Товар' 'Комплектующая'
#         when pg.id_payment is not null then 'Подарочная карта'
#         when a.correct_name ilike '%оставка%' then 'Доставка'
#         when os2.type is not null then 'Услуга'
#         when a.state != orders.item_state('BUNDLED') then 'Товар'
#         when a.state = orders.item_state('BUNDLED') then 'Комплектующая'
#         end "Тип позиции"
        
# --	,a.reference_id	"Товарная позиция" 
# --	,oplatforms.id 
# --  ,a.user_login
# --  ,a.in_account_date

# from PARAMS _
# cross join opentech.account_item ai
# /*Сайт старый новый, Маркетплейс, Платформа, Площадка и тп...*/
# left join orders.platforms oplatforms on oplatforms.id = ai.id_platform
# left join orders.states state on state.id = ai.state
# left join opentech.accounts a on a.id = ai.account_id
# left join opentech.customers c on c.id=a.id_customer
# left join opentech.store os ON a.store_id = os.id
# /* место продажи */
# join opentech.storage storage on storage.id = a.storage_id
# join public.cities pc on pc.id = storage.city_id 
# left join store.shipping_states sold_state ON sold_state.id = os.sold
# /*тип позиции (фон)*/
# left join payments.giftcards pg on a.i = pg.id_order_item 
# left join opentech.orderservices os2 on  a.i = os2.orderitemid
# left join orders.item_states ois on ois.id = a.state
# /*left join opentech.users u on lower(u.login) = lower(a.user_login)*/

# where
#     oplatforms.id in ( 7832, 1246, 3252, 2576, 5804) --3921	ASUP	Магазин
#     and ai.tm_placed::date between date_start and date_end --время Дата создания заказаия заказа
#     /*confluence*/
# --	and ai.tm_placed::date between '\$date_start' and '\$date_end' --время Дата создания заказаия заказа
# --	and case when 666=\$ordersatate then true else state.id=\$ordersatate end
#     /*jupyter*/
# --	ai.tm_placed::date between date_start and date_end --время Время оформления заказа	
#     /*tests*/
# --  and lai._values::hstore -> 'tm_placed' is not null 
# --  and a.state != 2 -- или a.del=0 /*это удалена или нет позиция лучше смотреть по состоянию a.state != 2*/
# --	ai.siteuserid is not null --учетная запись сайта, с которой был оформлен заказ
# --	and a.user_login = 'client' --кто добавил позицию в заказ
# --  and ai.state = 0 /*УДАЛЕННЫХ с ai.tm_placed = null ЕСТЬ!!*/
# --  and ai.tm_placed is null
# --	and 
# --	ai.account_id in ( 
# --		6273544, 6884837, 7109067, 7136854
# --		,
# --		7129622 , 7002268, 7140584
# ----		,7090797,7162886,7163571,7163728,7164250,7164450,7167963,7176968,7177077 
# --		)
# --group by 1,2,3,4,5,6,7,8, a.state
# --order by ai.account_id --, a.in_account_date
# )
# --select * from GENERAL

# , PAIDS as (
# select 
# 	pa.account_id 
# 	,array_to_string(array_agg(pa.payment_id),'; ') "ID платежей" 
# 	,array_to_string(array_agg(round(coalesce(case when op.type=payments.type('CASHLESS') and op.date_out is not null then -pa.sum else pa.sum end, 0), 2)), '; ') "Платежи по заказу"
# 	,coalesce(round(sum(case when op.type=payments.type('CASHLESS') and op.date_out is not null then -pa.sum else pa.sum end), 2), 0) "Оплачено по заказу"
# from opentech.payments_account pa 
# left join opentech.payments op on op.id = pa.payment_id
# where pa.account_id in (select distinct "ID заказа" from GENERAL )
# group by pa.account_id
# )
# --select * from PAIDS

# , PAIDS_ORDERS  as ( 
# select distinct
# --	*
# 	array_to_string(array_agg(distinct "Город отгрузки"),'; ') "Город отгрузки"
# 	,ge."Дата создания заказа"::text
# 	,ge."Время оформления заказа"::text		
# -- 	concat('<a href="http://opentech.local/new/order/',ge."ID заказа",'?main=2&additional=8">',ge."ID заказа",'</a>')  "ID заказа" /*confluence*/   
#     ,concat( '=HYPERLINK("http://opentech.local/new/order/',ge."ID заказа",'", "',ge."ID заказа",'")' )  "ID заказа" /*jupyter*/ 
# 	,ge."Cтатус заказа"
# 	,first_value( concat( '=HYPERLINK("http://opentech.local/user/', u.id, '", "', u.id, '")' )  
# 		) over (partition by ge."ID заказа" order by lai._values::hstore -> 'tm_placed' desc ) as "Оформил (id)" /*jupyter*/ 
# 	,first_value(u."name") over (partition by ge."ID заказа" order by lai._values::hstore -> 'tm_placed' desc ) as "Оформил"
# 	,first_value(lai._values::hstore -> 'tm_placed'::text) over (partition by ge."ID заказа" order by lai._values::hstore -> 'tm_placed' desc ) as "Время оформления заказа (logs)"
# 	,array_to_string(array_agg(distinct "Клиент (Покупатель)"),'; ') "Клиент (Покупатель)"
# 	,pa."ID платежей" 
# 	,pa."Платежи по заказу"	
# 	,pa."Оплачено по заказу"
#     ,ge."Стоимость оформленных позиций"
# 	,case when sum(ge.price_net) filter ( where a_state = 23 ) = 0 then null else sum(ge.price_net) filter ( where a_state = 23 ) end "Стоимость отгруженных позиций"
#     ,case when sum(ge.sprice) = 0 then null else sum(ge.sprice) end "Чистая себестоимость позиций" 
    
#     ,case /*% выкупа по оплате - стоимость оплаченных позиций/стоимость всех оформленных позиций*/
#         when "Оплачено по заказу" = any(array[null,0]) then null
#         when "Стоимость оформленных позиций" <> any(array[null,0]) /*деление на ноль*/
#             then round( "Оплачено по заказу" / "Стоимость оформленных позиций" * 100,0 )::text
#         else null end "% выкупа по оплате"
        
#     ,case /*% выкупа по отгрузке - стоимость отгруж позиций/стоимость всех оформленных позиций*/
#         when sum(ge.price_net) filter ( where a_state = 23 ) = any(array[null,0]) then null 
#         when "Стоимость оформленных позиций" <> any(array[null,0]) /*деление на ноль*/
#             then round(( sum(ge.price_net) filter ( where a_state = 23 ) / "Стоимость оформленных позиций") * 100,0)::text
#         else null end "% выкупа по отгрузке"	
	
#     ,sum(ge.price_net) filter ( where a_state = 23 ) - sum(ge.sprice) "Валовый доход по ЧС" -- сумма отгруженных позиций - чистая себестоимость = валовый доход по ЧС
    
#     ,array_to_string(array_agg(distinct "Площадка"),'; ') "Площадка"  
# 	,array_to_string(array_agg(distinct "Статусы списания товара ФОН"),'; ') "Статусы списания товара ФОН" --store.shipping_states Статусы списания товара
#     ,array_to_string(array_agg(distinct "Статус списания позиции"),'; ') "Статус списания позиции" -- Справочник состояний позиции клиентского заказа
#     ,array_to_string(array_agg(distinct "Тип позиции"),'; ') "Тип позиции"

# --	,lai._values::hstore -> 'tm_placed'
# --	,lai._values

# from GENERAL ge
# left join PAIDS pa on pa.account_id = ge."ID заказа"
# left join _logs.opentech__account_item lai on lai.id = ge.id
# left join opentech.users u on u.id = lai._id_user
# where 
# 	lai._values::hstore -> 'tm_placed' is not null
# group by 
# 	ge."ID заказа", ge."Дата создания заказа", ge."Время оформления заказа", ge."Cтатус заказа"
# 	,pa."ID платежей",pa."Платежи по заказу",pa."Оплачено по заказу",ge."Стоимость оформленных позиций" 
# 	,u."name", u.id, lai._values::hstore -> 'tm_placed'
# order by 3
# )

# select 
# 	row_number() OVER()::text "№" 
# 	,*
# from PAIDS_ORDERS

# union all
# select
# 	'ИТОГО' "№"
# 	,null "Город отгрузки", null "Дата создания заказа", null "Время оформления заказа", null "ID заказа", null "Cтатус заказа"
# 	,null "Оформил (id)", null "Оформил", null "Время оформления заказа (logs)"
# 	,null "Клиент (Покупатель)", null "ID платежей", null "Платежи по заказу"
# 	,round(sum("Оплачено по заказу"),2)
# 	,round(sum("Стоимость оформленных позиций"),2)
# 	,round(sum("Стоимость отгруженных позиций"),2)
# 	,round(sum("Чистая себестоимость позиций"),2)
# 	, null "% выкупа по оплате", null "% выкупа по отгрузке"
# 	,round(sum("Валовый доход по ЧС"),2)
# 	, null "Площадка", null "Статусы списания товара ФОН", null "Статус списания позиции", null "Тип позиции"
# from PAIDS_ORDERS
# group by 1,2,3,4,5,6,7,8,9,10,11,12, 17,18, 20,21,22,23


# '''

In [ ]:
sql_query_1 = '''

    -- BA-453 оптимизировал 

    /* 
     * BA-288 BA-453 Список оформленных заказов
     * https://confluence.e2e4.ru/pages/viewpage.action?pageId=183326070
     * https://jira.e2e4.ru/browse/BA-288
     * */

    with PARAMS as materialized (
        select 
--    		('01.03.2023' || ' 00:00:00' )::timestamp date_start 
--    		,('02.03.2023' || ' 23:59:59')::timestamp date_end
    --		 ( '\$date_start' || ' 00:00:00')::timestamp date_start 		/*confluence*/
    -- 		,( '\$date_end' || ' 23:59:59')::timestamp date_end 			/*confluence*/
            ( 'DATE_START_replce' || ' 00:00:00')::timestamp    date_start	/*jupyter*/
            ,( 'DATE_END_replce' || ' 23:59:59')::timestamp    date_end 		/*jupyter*/
        )

        
	, orders_logs as (
	select distinct 
		ai.account_id "id" --id - номер заказа,
		,max(lai._tm) over (partition by ai.account_id ) max_tm
		,max(lai._values::hstore -> 'tm_placed') over (partition by ai.account_id ) max_tm_placed
		,last_value(_id_user) over (partition by ai.account_id order by lai._tm) last_id_user
		
	from params _
	cross join opentech.account_item ai
	left join orders.platforms oplatforms on oplatforms.id = ai.id_platform
	--left join opentech.customers oc on oc.id = ai.id_customer
	/*logs*/
	left join _logs.opentech__account_item lai on lai.id = ai.id
	
	where 
--		oplatforms.id in ( 7832, 1246, 3252)
--		and 
		ai.id_customer not in (531058)
		and 
		lai._id_user in (1017, 112)
		and 
		lai._values::hstore -> 'tm_placed' is not null
		/*test*/
--		and 
--		account_id in (7313353)
	--	and yandex_client_id is not null
	)
	--select * from orders_logs os

    , ORDERS as ( 
    select distinct
        ai.id 
        ,a.i
        ,to_char(max_tm, 'YYYY-MM-DD HH24:MI:SS') "create_date_time как в 470"
        ,to_timestamp(max_tm_placed, 'YYYY-MM-DD HH24:MI:SS') "max_tm_placed Врем офор зак логи"
        ,max_tm_placed::timestamptz "z max_tm_placed Врем офор зак логи"

        ,ai.tm_created "Дата создания заказа не логи" --время Дата создания заказа
        ,ai.tm_placed "Дата оформления заказа не логи"
        /*,in_account_date "Время добавления позиции в заказ"*/
        ,ai.account_id "ID заказа" --позиции клиентских заказов
        ,lai.last_id_user
        ,ai.siteuserid "ID клиента siteuserid"
    --	,a.id a_id
        ,state.name "Cтатус заказа" -- orders.states справочник состояний клиентского заказа
    -- 	,case when c.id is null then null else
    --		concat('<a href="http://opentech.local/index.php?p=customers&id=',c.id,']">',c.name,'</a>') end "Клиент (Покупатель)" /*confluence*/
    --  ,c.id "ID клиента" /*jupyter*/ 
        ,c.name "Клиент (Покупатель)" /*jupyter*/
        ,pc.name "Город отгрузки"
        ,case orders.get_order_summa_brutto_without_logistics(ai.account_id) when 0 then null 
            else round(orders.get_order_summa_brutto_without_logistics(ai.account_id),2) end "Стоимость оформленных позиций"         
        ,oplatforms.name "Площадка" 
        ,sold_state.name "Статусы списания товара ФОН" --store.shipping_states Статусы списания товара
        ,ois.name "Статус списания позиции" -- Справочник состояний позиции клиентского заказа
        ,a.state a_state
        ,a.price_net price_net
        ,a.price
        ,os.sprice sprice
        ,case -- 'Подарочная карта' 'Доставка' 'Услуга' 'Товар' 'Комплектующая'
            when pg.id_payment is not null then 'Подарочная карта'
            when a.correct_name ilike '%оставка%' then 'Доставка'
            when os2.type is not null then 'Услуга'
            when a.state != orders.item_state('BUNDLED') then 'Товар'
            when a.state = orders.item_state('BUNDLED') then 'Комплектующая'
            end "Тип позиции"

    --	,a.reference_id	"Товарная позиция" 
    --	,oplatforms.id 
    --  ,a.user_login
    --  ,a.in_account_date

    from PARAMS _
    cross join opentech.account_item ai
    /*Сайт старый новый, Маркетплейс, Платформа, Площадка и тп...*/
    left join orders.platforms oplatforms on oplatforms.id = ai.id_platform
    left join orders.states state on state.id = ai.state
    left join opentech.accounts a on a.id = ai.account_id
    left join opentech.customers c on c.id=a.id_customer
    left join opentech.store os ON a.store_id = os.id
    /* место продажи */
    join opentech.storage storage on storage.id = a.storage_id
    join public.cities pc on pc.id = storage.city_id 
    left join store.shipping_states sold_state ON sold_state.id = os.sold
    /*тип позиции (фон)*/
    left join payments.giftcards pg on a.i = pg.id_order_item 
    left join opentech.orderservices os2 on  a.i = os2.orderitemid
    left join orders.item_states ois on ois.id = a.state
    /*left join opentech.users u on lower(u.login) = lower(a.user_login)*/
    /*logs*/
	join orders_logs lai on lai.id = ai.account_id


    where
--        oplatforms.id in ( 7832, 1246, 3252, 2576, 5804) --3921	ASUP	Магазин
--        and 
        max_tm_placed::timestamptz between date_start and date_end --время Дата создания заказаия заказа
        /*confluence*/
    --	and ai.tm_placed::date between '\$date_start' and '\$date_end' --время Дата создания заказаия заказа
    --	and case when 666=\$ordersatate then true else state.id=\$ordersatate end
        /*jupyter*/
    --	ai.tm_placed::date between date_start and date_end --время Время оформления заказа	
        /*tests*/
    --  and lai._values::hstore -> 'tm_placed' is not null 
    --  and a.state != 2 -- или a.del=0 /*это удалена или нет позиция лучше смотреть по состоянию a.state != 2*/
    --	ai.siteuserid is not null --учетная запись сайта, с которой был оформлен заказ
    --	and a.user_login = 'client' --кто добавил позицию в заказ
    --  and ai.state = 0 /*УДАЛЕННЫХ с ai.tm_placed = null ЕСТЬ!!*/
    --  and ai.tm_placed is null
--    	and 
--    	ai.account_id in ( 
--			7313353
--			)
    --group by 1,2,3,4,5,6,7,8, a.state
    --order by ai.account_id --, a.in_account_date
    )

    --select * from ORDERS
    --where "Cтатус заказа" ilike 'Удален'


    , PAIDS as (
    select 
        pa.account_id 
        ,array_to_string(array_agg(pa.payment_id),'; ') "ID платежей" 
        ,array_to_string(array_agg(round(coalesce(case 
	        	when op.type=payments.type('CASHLESS') and op.date_out is not null then -pa.sum else pa.sum end, 0), 2)), '; ') "Платежи по заказу"
        ,coalesce(round(sum(case 
	        	when op.type=payments.type('CASHLESS') and op.date_out is not null then -pa.sum else pa.sum end), 2), 0) "Оплачено по заказу"
    from opentech.payments_account pa 
    left join opentech.payments op on op.id = pa.payment_id
    left join opentech.account_item ai on pa.account_id = ai.account_id
    left join opentech.accounts oa on ai.account_id = oa.id

    where 
        pa.account_id in (select distinct "ID заказа" from ORDERS )
        and oa.del=0
        and oa.correct_name not like 'Доставка%' 

    group by pa.account_id
    )

    --select * from PAIDS 

    , PAIDS_ORDERS  as ( 
    select distinct
    --	*
        array_to_string(array_agg(distinct "Город отгрузки"),'; ') "Город отгрузки"
        ,os."Дата создания заказа не логи"::text
        ,os."Дата оформления заказа не логи"::text	
        
        ,os."create_date_time как в 470"::text
        ,os."max_tm_placed Врем офор зак логи"::text
        ,os."z max_tm_placed Врем офор зак логи"::text

        ,"ID клиента siteuserid"
    -- 	concat('<a href="http://opentech.local/new/order/',os."ID заказа",'?main=2&additional=8">',os."ID заказа",'</a>')  "ID заказа" /*confluence*/   
        ,concat( '=HYPERLINK("http://opentech.local/new/order/', os."ID заказа", '", "', os."ID заказа", '")' )  "ID заказа" /*jupyter*/ 
        ,os."Cтатус заказа"
        
/*
        ,first_value( concat( '=HYPERLINK("http://opentech.local/user/', u.id, '", "', u.id, '")' )  
            ) over (partition by os."ID заказа" order by lai._values::hstore -> 'tm_placed' desc ) as "Оформил (id)" /*jupyter*/ 
        ,first_value(u."name") over (partition by os."ID заказа" order by lai._values::hstore -> 'tm_placed' desc ) as "Оформил"
*/
        ,concat( '=HYPERLINK("http://opentech.local/user/', last_id_user, '", "', last_id_user, '")' ) "Оформил (id)" /*jupyter*/ 
        ,u.name "Оформил"
        
        ,array_to_string(array_agg(distinct "Клиент (Покупатель)"),'; ') "Клиент (Покупатель)"
    /*
        ,pa."ID платежей" 
        ,pa."Платежи по заказу"	
    */
        ,pa."Оплачено по заказу"
        ,os."Стоимость оформленных позиций"
        ,case when sum(os.price_net) filter ( where a_state = 23 and os."Тип позиции" not like 'Доставка%' ) = 0 then null 
            else sum(os.price_net) filter ( where a_state = 23 and os."Тип позиции" not like 'Доставка%' ) end "Стоимость отгруженных позиций"
        ,case when sum(os.sprice) filter ( where os."Тип позиции" not like 'Доставка%' ) = 0 then null 
            else sum(os.sprice) filter ( where os."Тип позиции" not like 'Доставка%' ) end "Чистая себестоимость позиций" 

    /*
        ,case /*% выкупа по оплате - стоимость оплаченных позиций/стоимость всех оформленных позиций*/
            when "Оплачено по заказу" = any(array[null,0]) then null
            when "Стоимость оформленных позиций" <> any(array[null,0]) /*деление на ноль*/
                then round( "Оплачено по заказу" / "Стоимость оформленных позиций" * 100,0 )::text
            else null end "% выкупа по оплате"

        ,case /*% выкупа по отгрузке - стоимость отгруж позиций/стоимость всех оформленных позиций*/
            when sum(os.price_net) filter ( where a_state = 23 ) = any(array[null,0]) then null 
            when "Стоимость оформленных позиций" <> any(array[null,0]) /*деление на ноль*/
                then round(( sum(os.price_net) filter ( where a_state = 23 ) / "Стоимость оформленных позиций") * 100,0)::text
            else null end "% выкупа по отгрузке"	
    */

        ,sum(os.price_net) filter ( where a_state = 23 ) - sum(os.sprice) "Валовый доход по ЧС" -- сумма отгруженных позиций - чистая себестоимость = валовый доход по ЧС

        ,array_to_string(array_agg(distinct "Площадка"),'; ') "Площадка"  
        ,array_to_string(array_agg(distinct "Тип позиции"),'; ') "Тип позиции"

    /*
        ,array_to_string(array_agg(distinct "Статусы списания товара ФОН"),'; ') "Статусы списания товара ФОН" --store.shipping_states Статусы списания товара
        ,array_to_string(array_agg(distinct "Статус списания позиции"),'; ') "Статус списания позиции" -- Справочник состояний позиции клиентского заказа
    */

    --	,lai._values::hstore -> 'tm_placed'
    --	,lai._values

    from ORDERS os
    left join PAIDS pa on pa.account_id = os."ID заказа"
--    left join _logs.opentech__account_item lai on lai.id = os.id
--    left join opentech.users u on u.id = lai._id_user
    left join opentech.users u on u.id = os.last_id_user

--    where 
--    	os."ID заказа" in ( 7313353 )
--        lai._values::hstore -> 'tm_placed' is not null

    group by 
        os."ID заказа", os."Дата создания заказа не логи", os."Дата оформления заказа не логи"
        ,os."create_date_time как в 470", os."max_tm_placed Врем офор зак логи" ,os."z max_tm_placed Врем офор зак логи"
        , "ID клиента siteuserid", os."Cтатус заказа", last_id_user
        ,pa."ID платежей",pa."Платежи по заказу",pa."Оплачено по заказу",os."Стоимость оформленных позиций" 
        ,u."name", u.id
--        , lai._values::hstore -> 'tm_placed'

    order by 3
    )

    select 
        row_number() OVER()::text "№" 
        ,*
    from PAIDS_ORDERS
    

    --union all
    --select
    --	'ИТОГО' "№"
    --	,null "Город отгрузки", null "Дата создания заказа", null "Время оформления заказа", null "ID заказа", null "Cтатус заказа"
    --	,null "Оформил (id)", null "Оформил", null "Время оформления заказа (logs)"
    --	,null "Клиент (Покупатель)", null "ID платежей", null "Платежи по заказу"
    --	,round(sum("Оплачено по заказу"),2)
    --	,round(sum("Стоимость оформленных позиций"),2)
    --	,round(sum("Стоимость отгруженных позиций"),2)
    --	,round(sum("Чистая себестоимость позиций"),2)
    --	, null "% выкупа по оплате", null "% выкупа по отгрузке"
    --	,round(sum("Валовый доход по ЧС"),2)
    --	, null "Площадка", null "Статусы списания товара ФОН", null "Статус списания позиции", null "Тип позиции"

'''

## Задаем параметры при необходимости

In [ ]:
'''Python OOP set widgets (for set params)'''

# print info (set params)
Prints_().print_set_params()

# ДАТЫ
DATE_START_replce = date.today() - relativedelta(months=1)
DATE_END_replce = date.today()

# значения дат по дефолту устанавливаем которые меняем с помощью пикера! Код ниже ВЫНОСИТЬ НИКУДА НЕ НАДО ИНАЧЕ ПОЕДУТ ЦИФРЫ! поэтому класс ниже через exec!!!
exec(MainVariables().exec_class_DatePicker)

# БРЕНДЫ
df_brand_val = 0 
# значение начальное которое меняем с помощью пикера! Код ниже ВЫНОСИТЬ НИКУДА НЕ НАДО ИНАЧЕ ПОЕДУТ ЦИФРЫ! поэтому класс ниже через exec!!!
exec(MainVariables().exec_class_BrandPicker)


df_for_brand_picker = None
# функция отображения всех заданных здесь наших виджетов
def display_all_set_params_widgets():
    global df_for_brand_picker
    '''display all widgets'''
#     date start, date end
    date_picker1 = DatePicker().dp_start( description = MainVariables().date_picker_description[0], DATE_START_replce = DATE_START_replce )
    date_picker2 = DatePicker().dp_end( description = MainVariables().date_picker_description[1], DATE_END_replce = DATE_END_replce )
#     brand
    df_for_brand_picker = ConnectDatabase().root_database_read_sql_query(MainVariables().sql_query_brand)
    brand_picker2 = BrandPicker().brand_picker(df_for_brand_picker)
    # display( d1 ), display( d2 ), display( bp )
    
    # колонка с датами VBox date_box
    date_box = widgets.VBox([ 
        date_picker1
        ,date_picker2
        ])
    box2 = widgets.VBox([ 
        brand_picker2 
    ])
    all_VBox_widgets = widgets.HBox([ date_box
#                                      , box2 
                                    ])
    display(all_VBox_widgets)
#     print(DATE_START_replce, DATE_END_replce )

display_all_set_params_widgets()

## Жмем кнопку запуска и ждем...

In [ ]:
'''Python OOP run button function widgets'''

all_sql_query_with_connect= { sql_query_1: 'root_database_read_sql_query'
#                       ,sql_query_2: 'root_database_read_sql_query'
                    }



class MakeOutputFile(FileName, ConnectDatabase, MainVariables):
    def make_name_for_output_file(self, ):
        s3_file_name = self.s3report_file_name( 
            ipynb_file_name .split('.')[0]
            ,getpass_getuser = getpass.getuser()
            ,datetime_now = datetime.datetime.now().strftime("%H:%M:%S")
            ,DATE_START_replce = DATE_START_replce
            ,DATE_END_replce = DATE_END_replce
            ,extension='.xlsx'
            ,params = df_for_brand_picker .loc[df_brand_val][1]
            )
        return s3_file_name
    
    exec(MainVariables().exec_def_sql_replace_params_class_MakeOutputFile)
        
    exec(MainVariables().exec_def_load_sql_into_df_class_MakeOutputFile)
        
    def make_excel_sheets_dict(self, dfs: list) -> dict: 
        try:
            # определяем названия листов в соответствии с данными на эти листы (листы в екселе)
            sheet_name0 = 'Итоговая'
            sheets0 = dfs[0]
            # СЛОВАРЬ названия листов и их данных 
            excel_sheets = {
                f'1 - {sheet_name0}' : sheets0, 
                }
            return excel_sheets
        except Exception as exc:
                print( Prints_().print_err_make_excel_sheets_dict(), exc )

# if we have changeable parameters, replace their into the sql query and print! for exemple for test on DBeaver 
# print( MakeOutputFile().sql_replace_params(all_sql_query_list, all_widget_in_this_report_replace_param)[0]   )

# 
exec( MainVariables().exec_class_RunFunction .replace('brand = ', '') ) # нету бренда
#     
exec(MainVariables().exec_class_RunClearButtons)
RunClearButtons().display_run_clear_button_widgets()   

pd.options.display.float_format = '{:.0f}'.format 
pd.set_option('max_rows', 500) 


In [ ]:
# ### if we have changeable parameters, replace their into the sql query and print! for exemple for test on DBeaver 
# ### dict params for replace 
# eval(MainVariables().all_replace_params_for_widgets) 

# ### print file name 
# print( MakeOutputFile().make_name_for_output_file() ) 

# ### print coonect database & sql 
# print( 
#     list(MakeOutputFile().sql_replace_params(all_sql_query_with_connect,  eval(MainVariables().all_replace_params_for_widgets)).values()) [0]
#     ,' - coonect to database\n'
#     ,list(MakeOutputFile().sql_replace_params(all_sql_query_with_connect,  eval(MainVariables().all_replace_params_for_widgets)).keys()) [0]
#     )

# Конец отчета

### Заметки!

In [ ]:
# info for users
try:
    Prints_().print_info4users( )
except:
    pass
